[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/atmos-cloud-sim-uj/PySDM-examples.git/main?urlpath=PySDM_examples/Lowe_et_al_2019/fig_3.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atmos-cloud-sim-uj/PySDM-examples/blob/main/PySDM_examples/Lowe_et_al_2019/fig_3.ipynb)

#### based on Fig. 3 from Lowe et al. 2019 (Nature Comm.)  "_Key drivers of cloud response to surface-active organics_"   
https://doi.org/10.1038/s41467-019-12982-0

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install PySDM-examples \
      notebook==`python -c "import notebook; print(notebook.__version__)"` \
      tornado==`python -c "import tornado; print(tornado.version)"`
    !ldconfig

In [2]:
from PySDM_examples.Lowe_et_al_2019 import Settings, Simulation
from PySDM_examples.Lowe_et_al_2019.aerosol import AerosolMarine, AerosolBoreal
from PySDM_examples.utils.show_plot import show_plot
from PySDM.physics.spectra import Lognormal
from PySDM.initialisation import spectral_sampling as spec_sampling
from PySDM.physics import si
import numpy as np
from matplotlib import pyplot

In [23]:
updraft_list = np.geomspace(0.1,10,2)
forg_list = np.linspace(0.1,0.9,3)
subplot_list = ["a","b","c","d"]

output = {}

updraft_list = np.geomspace(0.1,10,5)
forg_list = np.linspace(0.1,0.9,7)
subplot_list = ["b"]

for model in ('bulk','film'):
    for subplot in subplot_list:
        print(subplot, model)
        for w in updraft_list:
            for Forg in forg_list:
                aerosol_list = {"a": AerosolMarine(Acc_Forg=Forg, Acc_N2=30), 
                                "b": AerosolMarine(Acc_Forg=Forg, Acc_N2=134), 
                                "c": AerosolBoreal(Acc_Forg=Forg, Acc_N2=160), 
                                "d": AerosolBoreal(Acc_Forg=Forg, Acc_N2=540)}
                aerosol = aerosol_list[subplot]
                key = subplot+"_w{:.2f}_f{:.2f}_".format(w,Forg)+model
                #print(key)
                settings = Settings(
                    dt=1*si.s, n_sd_per_mode=128, 
                    model=model,
                    aerosol=aerosol,
                    w=w * si.m / si.s,
                    spectral_sampling=spec_sampling.ConstantMultiplicity
                )
                settings.output_interval = 2 * si.s
                simulation = Simulation(settings)
                output[key] = simulation.run()
                output[key]['updraft'] = w
                output[key]['org_fraction'] = Forg
                output[key]['color'] = aerosol.color

[0.1        0.23333333 0.36666667 0.5        0.63333333 0.76666667
 0.9       ]
b bulk
b film


0.1 0.1 98.32507386160411 107.69594862747682 9.53050366284244
0.1 0.23333333333333334 94.187685201959 110.79899331200087 17.636390653856285
0.1 0.3666666666666667 91.79479381281259 111.123180591605 21.056081696971567
0.1 0.5 88.36755476434831 114.9363695422816 30.066255481194336
0.1 0.6333333333333333 83.90596917347129 112.86767101257449 34.51685514677313
0.1 0.7666666666666666 78.41003581716872 107.04753558461898 36.52274797351869
0.1 0.9 69.81106032639137 91.59408005248733 31.20282033284217
0.31622776601683794 0.1 142.5476079627042 164.55288820831765 15.437144516217243
0.31622776601683794 0.23333333333333334 141.20656585364358 165.5739077669879 17.256521866412605
0.31622776601683794 0.3666666666666667 139.865523640026 166.5949270835354 19.110787810942792
0.31622776601683794 0.5 137.50346055111655 167.61594633375148 21.899438502815496
0.31622776601683794 0.6333333333333333 135.84239667565905 167.29592364659376 23.15442582041158
0.31622776601683794 0.7666666666666666 130.41727083222187

HTML(value="<a href='./tmpgmm3_j9l.pdf' target='_blank'>./tmpgmm3_j9l.pdf</a><br>")

In [24]:
for subplot in subplot_list:
    dCDNC = np.zeros((len(updraft_list), len(forg_list)))
    for i,w in enumerate(updraft_list):
        for j,Forg in enumerate(forg_list):
            key = subplot+"_w{:.2f}_f{:.2f}_".format(w,Forg)
            var = 'n_c_cm3'
            z = np.array(output[key+"film"]['z'])
            wz = np.where(z == z[-1])[0][0]
            CDNC_film = np.array(output[key+"film"][var])[wz]
            CDNC_bulk = np.array(output[key+"bulk"][var])[wz]
            dCDNC[i,j] = (CDNC_film - CDNC_bulk) / CDNC_bulk * 100.0
            print(w, Forg, CDNC_bulk, CDNC_film, dCDNC[i,j])

    if subplot == "a" or subplot == "b":
        pyplot.contour(forg_list, updraft_list, dCDNC, levels=[10,25], colors=["#1fa8f2","#4287f5"], linestyles=[":","--"], linewidths=4)
        pyplot.contourf(forg_list, updraft_list, dCDNC, cmap="Blues", levels=np.linspace(0,90,11))
        if subplot == "a":
            pyplot.title("MA Accum. mode conc. N$_2 = 30$ cm$^{-3}$")
        else:
            pyplot.title("MA Accum. mode conc. N$_2 = 135$ cm$^{-3}$")
    if subplot == "c" or subplot == "d":
        pyplot.contour(forg_list, updraft_list, dCDNC, levels=[10,25], colors=["#04c753","#157d3f"], linestyles=[":","--"], linewidths=4)
        pyplot.contourf(forg_list, updraft_list, dCDNC, cmap="Greens", levels=np.linspace(0,65,11))
        if subplot == "c":
            pyplot.title("HYY Accum. mode conc. N$_2 = 160$ cm$^{-3}$")
        else:
            pyplot.title("HYY Accum. mode conc. N$_2 = 540$ cm$^{-3}$")

    pyplot.title(subplot,weight="bold",loc="left")
    pyplot.xlabel("Organic mass fraction")
    pyplot.yscale("log")
    pyplot.yticks([0.1,1,10],["0.1","1","10"])
    pyplot.ylabel("Updraft [ms$^{-1}$]")
    pyplot.colorbar(label="$\Delta_{CDNC}$ [%]")
    show_plot()